In [1]:
import pandas as pd
from pandas import TimeSeries
import numpy as np
import math
from bokeh.charts import TimeSeries, output_file, show
import statsmodels.tsa.stattools as stat

from scipy import stats
import matplotlib.pyplot as plt
from math import *
from datetime import datetime
from datetime import timedelta

%matplotlib inline

In [2]:
sp500=pd.read_csv('sp_500_daily_prices.csv',usecols=['date','COMNAM','PRC',\
                    'RET','ASK','BID'],engine='c')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def data_process(start,period):
    sp500=pd.read_csv('sp_500_daily_prices.csv',usecols=['date','COMNAM','PRC',\
                    'RET','ASK','BID'],engine='c')
    date=sorted(list(set(sp500['date'])))
    if 10*period<len(date):
        train_start=date[-9*period+date.index(start)]
    sp_recent=sp500[sp500['date']>=train_start]
    sp_recent=sp_recent[sp_recent['date']<=date[date.index(start)+period]]
    
    # date_list=sorted(list(sp_recent['date']))
    company_recent=list(sp_recent[sp_recent['date']==start]['COMNAM'])
    sp_orig=sp_recent[sp_recent['COMNAM']==company_recent[0]][['date','RET']]
    sp_orig=pd.DataFrame(list(sp_orig['RET']),columns=[company_recent[0]],index=list(sp_orig['date']))
    sp_orig=sp_orig[~sp_orig.index.duplicated(keep='first')]
    for i in range(1,len(company_recent)):
        company=company_recent[i]
        current_data=sp_recent[sp_recent['COMNAM']==company][['date','RET']]
        ret=pd.DataFrame(list(current_data['RET']),columns=[company],index=list(current_data['date']))
        ret =ret[~ret.index.duplicated(keep='first')]
        sp_orig=pd.concat([sp_orig,ret],join='outer',axis=1)
    sp_ret=sp_orig.loc[train_start:start,:]
    sp_ret=sp_ret.dropna(axis=1)
    
    
    '''
    #sp_price=sp_recent[sp_recent['COMNAM']==company_recent[-1]][['date','PRC']]
    #sp_price=pd.DataFrame(list(sp_price['PRC']),columns=[company_recent[-1]],index=list(sp_price['date']))
    ##
    for company in company_recent:
        current_data=sp_recent[sp_recent['COMNAM']==company][['date','PRC']]
        price=pd.DataFrame(list(current_data['PRC']),columns=[company],index=list(current_data['date']))
        price =price[~price.index.duplicated(keep='first')]
        sp_price=pd.concat([sp_price,price],join='outer',axis=1)
    sp_price=sp_price.loc[train_start:start,:]
    '''
    
    #print('data_processing_done')
    return sp_recent,sp_ret,date

In [4]:
def filter_pair(pairs):
    selected_pair=[]
    stock_list=[]
    for pair in pairs:
        if pair[0] not in stock_list and pair[1] not in stock_list:
            selected_pair+=[pair]
            stock_list+=pair
            
    #print('filter_pair_done')
    return selected_pair

In [5]:
def find_pair(start,end):
    sp_recent,sp_ret,date=data_process(start,period)
    error_return=['E','D','C','B','A','']
    sp_ret=sp_ret.applymap(lambda x:np.float64(0.0) if x in error_return else np.float64(x))
    sp_pca=np.array(sp_ret)
    cov_company=np.cov(sp_pca)
    cov_company[np.isnan(cov_company)] = 0.0
    eigen_val,eigen_vector=np.linalg.eig(cov_company)
    eigen_vector_max=list(zip(eigen_vector[np.argmax(eigen_val)],sp_ret.columns))
    eigen_vector_max.sort(key=lambda x:x[0],reverse=True)
    eigen_vector_max=list(zip(*eigen_vector_max))
    data=sp_ret[list(eigen_vector_max[1][0:20])]
    score_matrix, pvalue_matrix, pairs=find_cointegrated_pairs(data)
    selected_pair=filter_pair(pairs)
    data_inuse=sp_recent[sp_recent['date']>=date[-period+date.index(start)]]
    data_inuse=data_inuse[data_inuse['date']<date[date.index(start)+period]]
    
    #print('find_pair_done')
    return data_inuse,selected_pair

In [6]:
def find_cointegrated_pairs(data):
    n = len(data.columns)
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = list(data.columns)
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = np.array(data[keys[i]])
            S2 = np.array(data[keys[j]])
            result = stat.coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.01:
                pairs.append((keys[i], keys[j]))
    
    #print('find_cointegrated_pairs_done')
    return score_matrix, pvalue_matrix, pairs

In [7]:
start=20130102
end=20150101
period=60

In [8]:
data,pairs=find_pair(start,period)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
pairs

[('WELLS FARGO & CO NEW', 'A E S CORP'),
 ('APARTMENT INVESTMENT & MGMT CO', 'M G I C INVESTMENT CORP WIS'),
 ('S P X CORP', 'JPMORGAN CHASE & CO'),
 ('DEVRY INC DEL', 'HALLIBURTON COMPANY'),
 ('OFFICEMAX INC NEW', 'BALL CORP'),
 ('TELLABS INC', 'MCDERMOTT INTERNATIONAL INC'),
 ('LOCKHEED MARTIN CORP', 'P M C SIERRA INC'),
 ('TITANIUM METALS CORP', 'BANK OF AMERICA CORP'),
 ('NORFOLK SOUTHERN CORP', 'PROGRESSIVE CORP OH'),
 ('STAPLES INC', 'AUTODESK INC')]

In [21]:
data

,date,COMNAM,PRC,RET,BID,ASK
12443,20121003,ORACLE CORP,31.8200,0.005371,31.80,31.81
12444,20121004,ORACLE CORP,31.8950,0.002357,31.90,31.91
12445,20121005,ORACLE CORP,31.3900,-0.015833,31.39,31.40
12446,20121008,ORACLE CORP,31.1800,-0.00669,31.17,31.18
12447,20121009,ORACLE CORP,30.6500,-0.016998,30.63,30.64
12448,20121010,ORACLE CORP,30.5800,-0.000326,30.58,30.59
12449,20121011,ORACLE CORP,30.7200,0.004578,30.72,30.73
12450,20121012,ORACLE CORP,31.0000,0.009115,31.00,31.01
12451,20121015,ORACLE CORP,31.3000,0.009677,31.29,31.30
12452,20121016,ORACLE CORP,31.8700,0.018211,31.86,31.87


In [8]:
signal_exists = ['Y','N']         # exist, not exist
signal_opens  = ['O', 'C']        # open, close
signal_moves  = ['B', 'S']        # buy, sell

In [9]:
def generate_signal(priceA, priceB):
    '''Find the trading signal in one window, default as 60 days'''
    
    # test the existence of signal
    tfA = len(np.unique(np.isnan(priceA)))
    tfB = len(np.unique(np.isnan(priceB)))
            
    if  tfA == 2 or tfB == 2:
        signal_exist = signal_exists[0]
    else:
        signal_exist = signal_exists[1]

        # calculate return for two stocks
        Areturn = np.diff(priceA)/priceA[:(len(priceA)-1)]
        Breturn = np.diff(priceB)/priceB[:(len(priceB)-1)]

        # regression on return
        beta, beta0, r_value, p_value, std_err = stats.linregress(Areturn, Breturn)

        # get the residual epsilon_t
        e_t = np.array(Breturn - beta0 - beta*Areturn)

        # auxiliary process X_t
        Xt = []
        for i in range(len(Areturn)):
            Xt.append(np.sum(e_t[:i+1]))

        # regression on X_t
        length = len(Xt)
        Xt_vec = np.array(Xt)
        b, a, r_value_x, p_value_x, std_err_x = stats.linregress(Xt_vec[1:],Xt_vec[:length-1])

        # get the residual zeta_t
        z_t = Xt_vec[:length-1] - a - b*Xt_vec[1:]
        var_z = np.var(z_t)

        # calculate the s-score
        s_score = -a*sqrt(1-b**2)/((1-b)*sqrt(var_z)) + int(a/(1-b))*sqrt((1-b**2)/var_z)

        # trading signal
        if s_score   < -1.25:
            signal_open = signal_opens[0]
            signal_move = signal_moves[0]
        elif s_score > 1.25:
            signal_open = signal_opens[0]
            signal_move = signal_moves[1]
        elif s_score < 0.5:
            signal_open = signal_opens[1]
            signal_move = signal_moves[0]
        elif s_score > -0.5:
            signal_open = signal_opens[1]
            signal_move = signal_moves[1]
        else:
            print('warning!')

        return beta,signal_exist, signal_open, signal_move

In [10]:
#position_dict = {}  # static variable: 'StockA': position
#cash = 50000        # static variable


def build_position(position_dict, cash, pairs, data):
    '''build position for each window
    
    Default parameters
    ------------------
    shares per trade:1000
    transaction cost fee: 0.0005
    '''
    # borrow cost of stock = 0.05 of short position
    borrow_cost = 0.05
    transaction_cost = 0.003
    PNL = []
    
    for i in range(60):
        cash_old = cash
        for pair in pairs:
            stockA         = data[data['COMNAM'] == pair[0]]
            stockB         = data[data['COMNAM'] == pair[1]]
            priceA         = pd.DataFrame.as_matrix(stockA['PRC'])
            priceB         = pd.DataFrame.as_matrix(stockB['PRC'])
            tickerA        = pair[0]
            tickerB        = pair[1]
            BidA           = pd.DataFrame.as_matrix(stockA['BID'])
            AskA           = pd.DataFrame.as_matrix(stockA['ASK'])
            BidB           = pd.DataFrame.as_matrix(stockB['BID'])
            AskB           = pd.DataFrame.as_matrix(stockB['ASK'])
            current_priceA = priceA[i:i+60]
            current_priceB = priceB[i:i+60]
            result         = generate_signal(current_priceA, current_priceB)
            beta           = result[0]
            open_or_not    = result[2]
            buy_or_sell    = result[3]
            
            # for A
            if (tickerA in position_dict.keys()): 
                if open_or_not == 'C':
                    if buy_or_sell == 'B':
                        cash -= (AskA[i+60]+transaction_cost)*abs(position_dict[tickerA])
                        position_dict[tickerA] = 0
                    else:
                        cash += (BidA[i+60]-transaction_cost)*abs(position_dict[tickerA])
                        position_dict[tickerA] = 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerA] -= int((cash/len(pairs))/AskB[i+60]*beta)
                        cash += (1-borrow_cost)*(cash/len(pairs))/AskB[i+60]*beta*(BidA[i+60]-transaction_cost)
                    else:
                        position_dict[tickerA] += int((cash/len(pairs))/BidB[i+60]*beta)
                        cash -= (cash/len(pairs))/BidB[i+60]*beta*(AskA[i+60]+transaction_cost)
            else:
                if open_or_not == 'C':
                    position_dict[tickerA] = 0
                    cash += 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerA] = -int((cash/len(pairs))/AskB[i+60]*beta)
                        cash += (1-borrow_cost)*abs(position_dict[tickerA])*(BidA[i+60]-transaction_cost)
                    else:
                        position_dict[tickerA] = int((cash/len(pairs))/BidB[i+60]*beta)
                        cash -= abs(position_dict[tickerA])*(AskA[i+60]+transaction_cost)
            
            # for B
            if (tickerB in position_dict.keys()):
                if open_or_not == 'C':
                    if buy_or_sell == 'B':
                        cash += (BidB[i+60]-transaction_cost)*abs(position_dict[tickerB])
                        position_dict[tickerB] = 0
                    else:
                        cash -= (AskB[i+60]+transaction_cost)*abs(position_dict[tickerB])
                        position_dict[tickerB] = 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerB] += int((cash/len(pairs))/AskB[i+60])
                        cash -= (cash/len(pairs))/AskB[i+60]*(AskB[i+60]+transaction_cost)
                    else:
                        position_dict[tickerB] -= int((cash/len(pairs))/BidB[i+60])
                        cash += (1-borrow_cost)*(cash/len(pairs))/BidB[i+60]*(BidB[i+60]-transaction_cost)
            else:
                if open_or_not == 'C':
                    position_dict[tickerB] = 0
                    cash += 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerB] = int((cash/len(pairs))/AskB[i+60])
                        cash -= abs(position_dict[tickerB])*(AskB[i+60]+transaction_cost)
                    else:
                        position_dict[tickerB] = int((cash/len(pairs))/BidB[i+60])
                        cash += (1-borrow_cost)*abs(position_dict[tickerB])*(BidB[i+60]-transaction_cost)
        PNL.append(cash - cash_old)    
    return position_dict, cash, PNL

In [79]:
build_position(position_dict, cash, pairs, data)

({'APARTMENT INVESTMENT & MGMT CO': 0,
  'BALL CORP': 0,
  'C A INC': 0,
  'CIENA CORP': -20,
  'CONAGRA INC': 875,
  'FRONTIER COMMUNICATIONS CORP': 0,
  'HALLIBURTON COMPANY': 0,
  'HARRIS CORP': 0,
  'HESS CORP': 0,
  'JOHNSON & JOHNSON': -181,
  'LOCKHEED MARTIN CORP': 0,
  'MOLEX INC': 0,
  'NEWMONT MINING CORP': 0,
  'NORFOLK SOUTHERN CORP': 434,
  'OWENS ILL INC': 0,
  'PROGRESSIVE CORP OH': 0,
  'ROCKWELL AUTOMATION INC': 583,
  'SYNOVUS FINANCIAL CORP': 0,
  'WELLS FARGO & CO NEW': 0,
  'X L GROUP PLC': -407},
 47627.987717273892,
 [9634.7454999999973,
  3849.4228913383195,
  9739.9338088117511,
  19078.378877723371,
  13175.744415521374,
  10534.792831208571,
  -4639.3725926757616,
  38364.016384874383,
  -41005.086384261318,
  3514.1515815765597,
  34594.270184904424,
  19388.744435475062,
  18577.469453275786,
  4580.9549435541267,
  -3911.0924537630635,
  -29313.697120454657,
  18008.121555588645,
  -3906.9887743889703,
  13332.586359062407,
  15440.118795314716,
  -41334.

In [11]:
def add_date(start_date, add_days):
    year = int(start_date/10000)
    month = int(start_date/100)- year*100
    day = start - year*10000 - month*100
    
    date1 = datetime(year, month, day)
    date2 = date1 + timedelta(days=add_days)
    
    new_date  = date2.year * 10000 + date2.month * 100 + date2.day
    return new_date
    

In [38]:
def backtest(start_date_index, timeline):
    date_list = sorted(list(set(sp500['date'])))
    win_len = 60
    start_date = date_list[start_date_index]
    pair_start_date = date_list[start_date_index -win_len*9]
    
    cash = 50000
    
    #pair_end_date = add_date(start_date, win_len - 1)
    
    
    PNL_list = []
    position_dict = {}
    
    for i in range(timeline):
        print("start: ", pair_start_date)
        data, pairs = find_pair(pair_start_date, win_len)
        position_dict, cash, PNL = build_position(position_dict, cash, pairs, data)
        PNL_list = PNL_list + PNL
        start_date_index += 60
        pair_start_date = date_list[start_date_index - win_len*9]
        print(cash)
    
    return PNL_list
      

In [39]:
start_date = 20130101
result = backtest(6000, 5)

start:  20110829


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  from ipykernel import kernelapp as app


13387.1606623
start:  20111122


ValueError: cannot convert float NaN to integer

In [37]:
date_list =sorted(list(set(sp500['date'])))
print(date_list[6000])
print(date_list[6060])

20131022
20140117
